# Решение задачи о назначениях
### Козловский А.А., гр. 2253

In [22]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import copy
import pandas as pd

from IPython.core.display import display, HTML
from IPython.display import display, Math, Latex, Markdown
import array_to_latex as a2l

## Описание необходимых функция для реализации венгерского алгоритма
### Алгоритмическая часть
#### Необходимые функции для алгоритмы Куна
В данном разделе описаны необходимые функции для реализации алгоритма Куна для поиска наибольшего паросочетания в двудольном графе

In [23]:
def dfs(v, graph, used, pars):
    """
    Алгоритм поиска в глубину для нахождения увеличивающей цепи
    
    graph - двудольный граф
    v - номер текущей вершины
    used - массив использованных вершин 1 доли
    pars - массив из номеров вершин первой доли, которые соответствуют вершинам 2 доли в паросочетании
    
    returns - есть или нет увеличивающая цепь
    """
    
    if used[v]:
        return False
    used[v] = True
    
    for index, value in enumerate(graph[v]):
        if value == 0:
            continue
            
        if pars[index] == -1 or dfs(pars[index], graph, used, pars):
            pars[index] = v
            return True
    
    return False

In [24]:
def kuhn_alg(graph):
    """
    Алгоритм Куна по нахождению наибольшего паросочетания
    
    graph - двудольный граф
    
    returns - массив из номеров вершин первой доли, 
    которые соответствуют вершинам 2 доли в наибольшем паросочетании
    """
    
    pars = np.full((graph.shape[1]), -1, dtype=int)
    
    for v, _ in enumerate(graph):
        used = np.zeros((graph.shape[0]), dtype=bool)
        dfs(v, graph, used, pars)
    
    return pars

### Дополнительные функции, необходимые для альфа-преобразования

In [25]:
def values_to_graph(values):
    """
    Функция строит матрицу смежности двудольного графа по нулям в матрице критериев
    
    values - матрица критериев
    
    returns - матрицу смежности двудольного графа
    """
    
    graph = np.zeros(values.shape, dtype=int)
    for ix, iy in np.ndindex(values.shape):
        if values[ix, iy] == 0:
            graph[ix, iy] = 1
    
    return graph        

In [26]:
def make_new_sets(graph, pars):
    """
    Алгоритм, который по чередующим цепям строит дополнительные множества для
    альфа-преобразования в матрице критериев для решения задачи о назначениях венгерским алгоритмом
    
    graph - матрица смежности двудольного графа
    pars - массив паросочетаний (индекс - вершина второй доли, значение элемента - вершина первой доли)
    
    returns - два отсортированных массива номеров вершин первой и второй долей
    (множества X' и Y' для альфа-преобразования)
    """
    x = []
    y = []
    
    for v, _ in enumerate(graph[:, 0]):
        used = np.zeros((graph.shape[0]), dtype=bool)
        if not v in pars:
            dfs_for_new_sets(v, graph, pars, used, x, y)
    
    x = np.array(list(set(x)), dtype=int)
    y = np.array(list(set(y)), dtype=int)
    
    return x, y

In [27]:
def dfs_for_new_sets(v, graph, pars, used, x, y):
    """
    Поиск в глубину, чтобы найти все вершины во всех возможных чередующихся цепях, чтобы построить
    дополнительные множества для альфа-преобразования
    
    v - номер текущей вершины первой доли
    graph - матрица смежности двудольного графа
    pars - массив паросочетаний (индекс - вершина второй доли, значение элемента - вершина первой доли)
    used - массив использованных вершин первой доли
    x - список пройденных вершин первой доли
    y - список пройденных вершин второй доли
    
    returns - true - если цепь идет дальше, false, если цепь закончилась
    """
    
    if used[v]:
        return
    used[v] = True
    x.append(v)
    
    for i, el in enumerate(graph[v]):
        if pars[i] == v or el == 0:
            continue
        if pars[i] == -1:
            return
        else:
            y.append(i)
            dfs_for_new_sets(pars[i], graph, pars, used, x, y)

In [28]:
def alpha_transformation(values, x, y):
    """
    Альфа-преобразование для матрицы критериев
    
    values - матрица критериев
    x, y - дополнительные множества
    """
    
    used_x = np.zeros((values.shape[0]), dtype=bool)
    used_y = np.zeros((values.shape[1]), dtype=bool)
    
    for el in x:
        used_x[el] = True
    for el in y:
        used_y[el] = True
        
        
    min_ = 1000
    for ix, iy in np.ndindex(values.shape):
        if used_x[ix] and not used_y[iy]:
            min_ = min(values[ix, iy], min_)
    
    
    for ix, iy in np.ndindex(values.shape):
        if used_x[ix]:
            values[ix, iy] -= min_
        if used_y[iy]:
            values[ix, iy] += min_

In [29]:
def min_to_max_task(values):
    """
    Функция для преборазований матрицы коэффициентов, чтобы матрица критериев стала для решения
    задачи на минимум, когда изначально задача была на максимум
    
    values - матрица критериев изначальной задачи на максимум (он не будет меняться)
    
    returns - новую матрицу критериев для задачи на минимум
    """
    
    new_values = values.copy()
    new_values *= -1
    new_values += np.min(new_values)
    
    return new_values

In [30]:
def to_close_type(values):
    """
    Функция приводит матрицу критериев к квадратному виду, чтобы задача стала закрытого типа
    
    values - матрица критериев
    
    returns - новая матрица критериев
    """
    
    n = max(values.shape[0], values.shape[1])
    new_values = np.zeros((n, n))
    
    for ix, iy in np.ndindex(values.shape):
        new_values[ix, iy] = values[ix, iy]
        
    return new_values

In [31]:
def norm(matrix):
    """
    Функция для нормирования матрицы
    
    matrix - входная матрица
    
    returns - матрицу с нормированными значениями"""
    matrix_copy = np.array(matrix, dtype=float)
    
    min_ = np.min(matrix)
    max_ = np.max(matrix)
    delta = max_ - min_
    matrix_copy -= min_
    
    for ix, iy in np.ndindex(matrix.shape):
        matrix_copy[ix, iy] /= delta
    
    return matrix_copy

In [32]:
def get_answers(pars, values_shape):
    """
    returns - Получает пары значений по x и y, которые надо брать для решения задачи из матрицы коэффициентов
    
    pars - массив паросочетаний
    values_shape - shape изначальной матрицы критериев
    """
    
    n = min(values_shape[0], values_shape[1])
    current = []
    for i, el in enumerate(pars):
        if i < values_shape[0] and el < values_shape[1]:
            current.append((el, i))
    
    return np.array(current)

In [33]:
def sum_for_pars(pars, values):
    """
    Функция для подсчета суммы из матрицы критериев по паросочетанию
    
    pars - массив паросочетаний
    values - матрица критерие
    
    returns - сумму элементов матрицы критериев по паросочетанию"""
    
    result = 0
    for j, i in enumerate(pars):
        if i < values.shape[0] and j < values.shape[1]:
            result += values[i, j]
        
    return result

### Венгерский алгоритм
Венгерский алгоритм. Основная идея в нахождении совершенного паросочетания наименьшего веса в двудольном графе. Первая доля - работники, вторая доля - вакансии (если мы рассматриваем задачу о назначении). Алгоритм может применяться и для других схожих задач

In [34]:
def hungarian_algorithm(values, is_min=True):
    """
    Венгерский алгоритм
    
    values - матрица критериев
    
    returns - сумма целевой функции, массив паросочетаний
    """
    
    values_initial = values.copy()  # Делаем копию массива критериев, чтобы потом посчитать сумму целевой функции
    values -= np.min(values)
    
    # Если задача на максимум, то преобразуем матрицу критериев
    if not is_min:
        values = min_to_max_task(values)
        
    graph = values_to_graph(values)  # Переводим матрицу критериев в матрицу смежности двудольного граф
    pars = kuhn_alg(graph)  # По алгоритму Куна находим наибольшее паросочетание
    
    # Цикл запускаем пока максимальное паросочетание не станет совершенным
    while -1 in pars:  
        x, y = make_new_sets(graph, pars)  # Строим дополнительные множества для альфа-преобразования
        alpha_transformation(values, x, y)  # Альфа преобразования
        graph = values_to_graph(values)  # Переводим новую матрицу критериев в матрицу смежности двудольного графа
        pars = kuhn_alg(graph)  # Находим наибольшее паросочетание по алгоритму Куна
        
    # Считаем результат по изначальной матрице критериев
    result = 0
    for i, par in enumerate(pars):
        result += values_initial[par, i]
 
    return result, pars

## Решение практической задачи
Для начала рассмотрим тестовый пример с матрице коэффициентов

$$\begin{pmatrix}
32 & 28 & 4 & 26 & 4\\
17 & 19 & 4 & 17 & 4\\
4 & 4 & 5 & 4 & 4\\
17 & 14 & 4 & 14 & 4\\
21 & 16 & 4 & 13 & 4
\end{pmatrix}$$
Ответ должен получиться 39

In [35]:
values = np.array([[32, 28, 4, 26, 4], 
                   [17, 19, 4, 17, 4], 
                   [4, 4, 5, 4, 4], 
                   [17, 14, 4, 14, 4], 
                   [21, 16, 4, 13, 4]], dtype=int)
result, pars = hungarian_algorithm(values)
print(result, pars)

39 [2 3 1 4 0]


Как мы видим ответ получается такой, какой и должен получаться

## Решение своей задачи

### Задача:
Разработка платформы по разработке векторного видео и сжатию данных. 
### Вакансии:
 
    1. Специалист по математической статистике и машинному обучению
    2. Специалист Computer Science, занимающийся теорией информации
    3. Прикладной математик
    4. Человек занимающийся распознаванием образов
    5. Специалист по алгоритмам
### Задачи:

    1. Разработка алгоритма для определения объектов на абстрактном видео (АООАВ)
    2. Разработка алгоритма для интерполяции или регрессии движения объектов на видео (АИРДОВ)
    3. Разработка алгоритма для отделения движения объектов и камеры на видео (ОДОК)
    4. Разработка алгоритмы для интерполяции или регрессии движения сцены на видео (
    5. Разработка алгоритма, использующего машинное обучение для нахождения паттернов в 0 и 1
    6. Разработка протокола по которым будут хранится метаданные
    7. Сформулировать и описать функции, которые будут использоваться для формирования метаданных
    8. Разработать алгоритм для определения метаданных по данным с использованием специальных функций
### Критерии:

    1. Соответствие между предметной областью специалиста и задачей (измеряется баллами от 0 до 10)
    2. Количество научных публикаций по теме, соответствующей или тесно связанной с предложенной задачей (измеряется кол-вом работ)


### Подготовка к решению задачи
В матрице критериев строками будут являться работники, а столбцами - задачи

Для начала стоит отметить, что данная задача является задачей открытого типа, так как матрица критериев не квадратная. Перед решением нам необходимо привести задачу к задаче закрытого типа. Для этого добавим дополнительные нулевые строки.  
### Начальные данные
Рассмотрим решение задачи относительно 2 критериев, по отдельности и вместе

<b><i>Критерии были выбраны следующие</i></b>:

    1. Количество опубликованных работ по тематике, связанной с задачей (позитивный критерий, измеряется в штуках)
    2. Требуемой количество часов на выполнения задания (негативный критерий, измеряется в часах)

### Решение задачи по первому критерию

Решим задачу по первому критерию. Это позитивный критерий, поэтому решаем задачу на максимум, т.е. находим в двудольном графе совершенное паросочетание наибольшего веса

In [36]:
workers = np.array(['Специалист по математической статистике и машинному обучению',
                   'Специалист Computer Science, занимающийся теорией информации',
                   'Прикладной математик',
                   'Человек занимающийся распознаванием образов',
                   'Специалист по алгоритмам'], dtype=str)
occupations = np.array(['Разработка алгоритма для определения объектов на абстрактном видео',
                        'Разработка алгоритма для интерполяции или регрессии движения объектов на видео',
                        'Разработка алгоритма для отделения движения объектов и камеры на видео',
                        'Разработка алгоритмы для интерполяции или регрессии движения сцены на видео',
                        'Разработка алгоритма, использующего машинное обучение для нахождения паттернов в 0 и 1',
                        'Разработка протокола по которым будут хранится метаданные',
                        'Сформулировать и описать функции, которые будут использоваться для формирования метаданных',
                        'Разработать алгоритм для определения метаданных по данным с использованием специальных функций'],
                      dtype=str)

values = np.array([[8, 9, 7, 9, 10, 1, 0, 1],
                  [3, 2, 0, 2, 3, 9, 10, 8],
                  [6, 8, 6, 8, 8, 7, 9, 8],
                  [10, 7, 10, 7, 1, 0, 0, 0],
                  [6, 7, 7, 6, 7, 6, 7, 8]], dtype=int)

df = pd.DataFrame(index=workers, data=values, columns=occupations)
html = df.to_html()

display(Markdown('### <i>Данные по первому критерию</i>'))
display(HTML(html))

### <i>Данные по первому критерию</i>

,Разработка алгоритма для определения объектов на абстрактном видео,Разработка алгоритма для интерполяции или регрессии движения объектов на видео,Разработка алгоритма для отделения движения объектов и камеры на видео,Разработка алгоритмы для интерполяции или регрессии движения сцены на видео,"Разработка алгоритма, использующего машинное обучение для нахождения паттернов в 0 и 1",Разработка протокола по которым будут хранится метаданные,"Сформулировать и описать функции, которые будут использоваться для формирования метаданных",Разработать алгоритм для определения метаданных по данным с использованием специальных функций
Специалист по математической статистике и машинному обучению,8,9,7,9,10,1,0,1
"Специалист Computer Science, занимающийся теорией информации",3,2,0,2,3,9,10,8
Прикладной математик,6,8,6,8,8,7,9,8
Человек занимающийся распознаванием образов,10,7,10,7,1,0,0,0
Специалист по алгоритмам,6,7,7,6,7,6,7,8


In [37]:
result, pars = hungarian_algorithm(to_close_type(values), False)

display(Markdown(f'<b>Ответ</b>: {np.round(result, 2)}, <br/> <b>Позиции</b>: {get_answers(pars, values.shape)}'))

<b>Ответ</b>: 46.0, <br/> <b>Позиции</b>: [[7 0]
 [6 1]
 [3 2]
 [5 3]
 [0 4]]

In [38]:
values_2 = np.array([[np.random.randint(5, 20) for i in range(8)] for j in range(5)], dtype=int)
df_2 = pd.DataFrame(index=workers, data=values_2, columns=occupations)
html = df_2.to_html()

display(Markdown('### <i>Данные по второму критерию</i>'))
display(HTML(html))

### <i>Данные по второму критерию</i>

,Разработка алгоритма для определения объектов на абстрактном видео,Разработка алгоритма для интерполяции или регрессии движения объектов на видео,Разработка алгоритма для отделения движения объектов и камеры на видео,Разработка алгоритмы для интерполяции или регрессии движения сцены на видео,"Разработка алгоритма, использующего машинное обучение для нахождения паттернов в 0 и 1",Разработка протокола по которым будут хранится метаданные,"Сформулировать и описать функции, которые будут использоваться для формирования метаданных",Разработать алгоритм для определения метаданных по данным с использованием специальных функций
Специалист по математической статистике и машинному обучению,13,12,10,14,8,7,16,18
"Специалист Computer Science, занимающийся теорией информации",16,13,15,11,14,14,13,18
Прикладной математик,17,6,9,19,17,11,10,11
Человек занимающийся распознаванием образов,5,14,5,14,16,14,16,13
Специалист по алгоритмам,16,9,18,13,8,6,19,5


In [39]:
result, pars = hungarian_algorithm(to_close_type(values_2))

display(Markdown(f'<b>Ответ</b>: {np.round(result, 2)}, <br/> <b>Позиции</b>: {get_answers(pars, values_2.shape)}'))
display(Markdown('<hr/>'))

<b>Ответ</b>: 34.0, <br/> <b>Позиции</b>: [[7 0]
 [2 1]
 [3 2]
 [1 3]
 [6 4]]

<hr/>

## Решение задачи по смешанному критерию
Рассмотрим решению задачи по смешанному критерию. Пусть матрица 1 критерия - <b>X</b>, а 2 критерия - <b>Y</b>, а смешанный критерий пусть определяет матрица <b>Z</b>. Отметим, 1 критерий - позитивный, а 2 - негативный и они имеют для нас одинаковую важность. Составим линейную выпуклую комбинацию для построения обобщенного критерия 
$$\begin{equation}
Z = AX - BY
\end{equation}$$

$$\begin{equation}
A + B = 1 
\end{equation}$$

Так как критерии для нас равносильны, то положим
$$\begin{equation}
A = B = \frac{1}{2}
\end{equation}$$

Но перед тем как находить матрицу смешанного критерия необходимо нормировать значения 1 критерия и 2. Для этого делаем очевидные действия

$$\begin{equation}
x_{i, j} = \frac{x_{i, j} - min(X)}{|max(X) - min(X)|}
\end{equation}$$

$$\begin{equation}
y_{i, j} = \frac{y_{i, j} - min(Y)}{|max(Y) - min(Y)|}
\end{equation}$$

Теперь значения первых двух критериев принимают значения от 0 до 1 и мы можем строить обощенный критерий, используя линейную выпуклую комбинацию, полученную нами ранее

### Необходимые расчеты

Для начала необходимо нормировать значения. Сделаем это и выведем результаты в таблицы

In [40]:
# Нормирование
norm_values = norm(values)
norm_values_2 = norm(values_2)

# Создание датафреймов по нормированным данным
norm_df_1 = pd.DataFrame(index=workers, data=norm_values, columns=occupations)
norm_df_2 = pd.DataFrame(index=workers, data=norm_values_2, columns=occupations)

# Перевод датафреймов в html код
html_norm_1 = norm_df_1.to_html()
html_norm_2 = norm_df_2.to_html()

# Вывод таблиц
display(Markdown('### Таблица нормированных оценок по 1 критерию'))
display(HTML(html_norm_1))

display(Markdown('### Таблица нормированных оценок по 2 критерию'))
display(HTML(html_norm_2))



values_3 = np.array([[norm_values[j, i] * 0.5 - norm_values_2[j, i] * 0.5 for i in range(norm_values.shape[1])]
                    for j in range(norm_values.shape[0])], dtype=float)

norm_df_3 = pd.DataFrame(index=workers, data=values_3, columns=occupations)
html_norm_3 = norm_df_3.to_html()

display(Markdown('### Таблица оценок по обобщенному критерию'))
display(HTML(html_norm_3))


### Таблица нормированных оценок по 1 критерию

,Разработка алгоритма для определения объектов на абстрактном видео,Разработка алгоритма для интерполяции или регрессии движения объектов на видео,Разработка алгоритма для отделения движения объектов и камеры на видео,Разработка алгоритмы для интерполяции или регрессии движения сцены на видео,"Разработка алгоритма, использующего машинное обучение для нахождения паттернов в 0 и 1",Разработка протокола по которым будут хранится метаданные,"Сформулировать и описать функции, которые будут использоваться для формирования метаданных",Разработать алгоритм для определения метаданных по данным с использованием специальных функций
Специалист по математической статистике и машинному обучению,0.8,0.9,0.7,0.9,1.0,0.1,0.0,0.1
"Специалист Computer Science, занимающийся теорией информации",0.3,0.2,0.0,0.2,0.3,0.9,1.0,0.8
Прикладной математик,0.6,0.8,0.6,0.8,0.8,0.7,0.9,0.8
Человек занимающийся распознаванием образов,1.0,0.7,1.0,0.7,0.1,0.0,0.0,0.0
Специалист по алгоритмам,0.6,0.7,0.7,0.6,0.7,0.6,0.7,0.8


### Таблица нормированных оценок по 2 критерию

,Разработка алгоритма для определения объектов на абстрактном видео,Разработка алгоритма для интерполяции или регрессии движения объектов на видео,Разработка алгоритма для отделения движения объектов и камеры на видео,Разработка алгоритмы для интерполяции или регрессии движения сцены на видео,"Разработка алгоритма, использующего машинное обучение для нахождения паттернов в 0 и 1",Разработка протокола по которым будут хранится метаданные,"Сформулировать и описать функции, которые будут использоваться для формирования метаданных",Разработать алгоритм для определения метаданных по данным с использованием специальных функций
Специалист по математической статистике и машинному обучению,0.571429,0.500000,0.357143,0.642857,0.214286,0.142857,0.785714,0.928571
"Специалист Computer Science, занимающийся теорией информации",0.785714,0.571429,0.714286,0.428571,0.642857,0.642857,0.571429,0.928571
Прикладной математик,0.857143,0.071429,0.285714,1.000000,0.857143,0.428571,0.357143,0.428571
Человек занимающийся распознаванием образов,0.000000,0.642857,0.000000,0.642857,0.785714,0.642857,0.785714,0.571429
Специалист по алгоритмам,0.785714,0.285714,0.928571,0.571429,0.214286,0.071429,1.000000,0.000000


### Таблица оценок по обобщенному критерию

,Разработка алгоритма для определения объектов на абстрактном видео,Разработка алгоритма для интерполяции или регрессии движения объектов на видео,Разработка алгоритма для отделения движения объектов и камеры на видео,Разработка алгоритмы для интерполяции или регрессии движения сцены на видео,"Разработка алгоритма, использующего машинное обучение для нахождения паттернов в 0 и 1",Разработка протокола по которым будут хранится метаданные,"Сформулировать и описать функции, которые будут использоваться для формирования метаданных",Разработать алгоритм для определения метаданных по данным с использованием специальных функций
Специалист по математической статистике и машинному обучению,0.114286,0.200000,0.171429,0.128571,0.392857,-0.021429,-0.392857,-0.414286
"Специалист Computer Science, занимающийся теорией информации",-0.242857,-0.185714,-0.357143,-0.114286,-0.171429,0.128571,0.214286,-0.064286
Прикладной математик,-0.128571,0.364286,0.157143,-0.100000,-0.028571,0.135714,0.271429,0.185714
Человек занимающийся распознаванием образов,0.500000,0.028571,0.500000,0.028571,-0.342857,-0.321429,-0.392857,-0.285714
Специалист по алгоритмам,-0.092857,0.207143,-0.114286,0.014286,0.242857,0.264286,-0.150000,0.400000


### Решение задачи

In [41]:
values_3 -= np.min(values_3)
result, pars = hungarian_algorithm(to_close_type(values_3), False)

pos = get_answers(pars, values_3.shape)

display(Markdown('### Ответ'))
display(Markdown('### <i>Оптимальные назначения</i>'))

for el in pos:
    display(Markdown(f'<b>{workers[el[1]]}</b>: {occupations[el[0]]}'))
    
display(Markdown('### <i>Результаты</i>'))
result_experience = sum_for_pars(pars, values)
result_salary = sum_for_pars(pars, values_2)

display(Markdown(f'<b>Суммарные баллы опыта</b>: {result_experience} у.е.'))
display(Markdown(f'<b>Затраченные деньги</b>: {result_salary} тыс. руб.'))

### Ответ

### <i>Оптимальные назначения</i>

<b>Специалист по математической статистике и машинному обучению</b>: Разработать алгоритм для определения метаданных по данным с использованием специальных функций

<b>Специалист Computer Science, занимающийся теорией информации</b>: Разработка алгоритма для отделения движения объектов и камеры на видео

<b>Прикладной математик</b>: Разработка алгоритмы для интерполяции или регрессии движения сцены на видео

<b>Человек занимающийся распознаванием образов</b>: Сформулировать и описать функции, которые будут использоваться для формирования метаданных

<b>Специалист по алгоритмам</b>: Разработка алгоритма для определения объектов на абстрактном видео

### <i>Результаты</i>

<b>Суммарные баллы опыта</b>: 46 у.е.

<b>Затраченные деньги</b>: 37 тыс. руб.

In [21]:
# Определяем столбцы и массив для данных для датафрейма. Массив индексов определен ранее - workers
result_df_columns = np.array(['Назначенная работа', 'Опыт (у. е.)', 'Требуемая зарплата (тыс. руб.)'])
result_df_data = np.zeros((workers.shape[0], result_df_columns.shape[0]), dtype=object)

df_result = pd.DataFrame(index=workers, data=result_df_data, columns=result_df_columns)

# Заполняем данные для датафрема
# Заполняем назначенные задачи
for el in pos:
    df_result.loc[workers[el[1]], result_df_columns[0]] = occupations[el[1]]
    df_result.loc[workers[el[1]], result_df_columns[1]] = values[el[1], el[0]]
    df_result.loc[workers[el[1]], result_df_columns[2]] = values_2[el[1], el[0]]

    
html_result_df = df_result.to_html()

display(Markdown('### Таблица назначений по смешанному критерию'))
display(HTML(html_result_df))

### Таблица назначений по смешанному критерию

,Назначенная работа,Опыт (у. е.),Требуемая зарплата (тыс. руб.)
Специалист по математической статистике и машинному обучению,Разработка алгоритма для определения объектов на абстрактном видео,1,16
"Специалист Computer Science, занимающийся теорией информации",Разработка алгоритма для интерполяции или регрессии движения объектов на видео,10,9
Прикладной математик,Разработка алгоритма для отделения движения объектов и камеры на видео,8,9
Человек занимающийся распознаванием образов,Разработка алгоритмы для интерполяции или регрессии движения сцены на видео,1,7
Специалист по алгоритмам,"Разработка алгоритма, использующего машинное обучение для нахождения паттернов в 0 и 1",6,8
